In [19]:
%load_ext autoreload
%autoreload 2
import sys
import os
sys.path.insert(0, os.path.abspath('../..'))
import numpy as np
import pandas as pd
# mcfly
from mcfly import modelgen, find_architecture, storage
from utils import tutorial_pamap2
import json

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
# Specify in which directory you want to store the data:
data_dir = "/media/sf_VBox_Shared/timeseries/PAMAP_Dataset/"
outputdir = os.path.join(data_dir, "cleaned_7act")
if not os.path.exists(outputdir):
    os.makedirs(outputdir)

In [21]:
columns_to_use = ['hand_acc_16g_x', 'hand_acc_16g_y', 'hand_acc_16g_z',
                          'ankle_acc_16g_x', 'ankle_acc_16g_y', 'ankle_acc_16g_z',
                          'chest_acc_16g_x', 'chest_acc_16g_y', 'chest_acc_16g_z']

In [22]:
def get_header_pamap():
    axes = ['x', 'y', 'z']
    IMUsensor_columns = ['temperature'] + \
        ['acc_' + i for i in axes] + \
        ['gyroscope_' + i for i in axes] + \
        ['magnometer_' + i for i in axes] + \
        ['orientation_' + str(i) for i in range(4)]
    header = ["timestamp", "activityID", "heartrate"] + ["hand_" + s
                                                         for s in IMUsensor_columns] \
        + ["chest_" + s for s in IMUsensor_columns] + ["ankle_" + s
                                                       for s in IMUsensor_columns]
    return header

len(get_header_pamap())

45

In [23]:
example_file_pamap = os.path.join(targetdir, 'PAMAP_Dataset/Indoor/subject1.dat')
example_pamap = pd.read_csv(example_file_pamap, header=None, sep=' ')
example_pamap.columns = get_header_pamap()

example_file_pamap2 = os.path.join('/media/sf_VBox_Shared/timeseries/PAMAP2_Dataset/Protocol/subject101.dat')
example_pamap2 = pd.read_csv(example_file_pamap2, header=None, sep=' ')
example_pamap2.columns = tutorial_pamap2.get_header()

In [24]:
example_pamap.shape, example_pamap2.shape

((198000, 45), (376417, 54))

Data format – each of the dat-files contains 45 columns per row – the columns contain the following data: 
– 1 timestamp (s) 
– 2 activityID (see below) 
– 3 heart rate (bpm) 
– 4-17 IMU hand 
– 18-31 IMU chest 
– 32-45 IMU shoe 
– IMU sensor data contains following columns: 
 – 1 temperature (°C) 
 – 2-4 3D-acceleration (ms-2) 
 – 5-7 3D-gyroscope (rad/s) 
 – 8-10 3D-magnetometer (μT) 
 – 11-14 orientation (turned off during this data collection)

In [25]:
example_pamap.head()

,timestamp,activityID,heartrate,hand_temperature,hand_acc_x,hand_acc_y,hand_acc_z,hand_gyroscope_x,hand_gyroscope_y,hand_gyroscope_z,...,ankle_gyroscope_x,ankle_gyroscope_y,ankle_gyroscope_z,ankle_magnometer_x,ankle_magnometer_y,ankle_magnometer_z,ankle_orientation_0,ankle_orientation_1,ankle_orientation_2,ankle_orientation_3
0,0.01,0.0,53.0,29.62,6.16,1.92,6.85,0.06,-0.03,-0.02,...,0.07,-0.04,-0.00,-18.28,-12.12,-25.55,1.0,0.0,0.0,0.0
1,0.02,0.0,53.0,29.62,6.20,1.92,6.81,0.01,-0.00,-0.03,...,0.05,-0.03,0.02,-18.28,-12.12,-25.55,1.0,0.0,0.0,0.0
2,0.03,0.0,53.0,29.62,6.24,1.77,6.69,0.03,0.01,-0.01,...,0.05,-0.04,0.02,-18.28,-12.12,-25.55,1.0,0.0,0.0,0.0
3,0.04,0.0,53.0,29.62,6.31,1.92,7.01,0.03,-0.03,-0.00,...,0.02,-0.03,0.00,-18.28,-12.12,-25.55,1.0,0.0,0.0,0.0
4,0.05,0.0,53.0,29.62,6.12,1.96,6.81,0.05,-0.11,0.02,...,0.04,-0.01,0.00,-18.27,-12.13,-27.27,1.0,0.0,0.0,0.0


In [26]:
example_pamap2.head()

,timestamp,activityID,heartrate,hand_temperature,hand_acc_16g_x,hand_acc_16g_y,hand_acc_16g_z,hand_acc_6g_x,hand_acc_6g_y,hand_acc_6g_z,...,ankle_gyroscope_x,ankle_gyroscope_y,ankle_gyroscope_z,ankle_magnometer_x,ankle_magnometer_y,ankle_magnometer_z,ankle_orientation_0,ankle_orientation_1,ankle_orientation_2,ankle_orientation_3
0,8.38,0,104.0,30.0,2.37223,8.60074,3.51048,2.43954,8.76165,3.35465,...,0.008300,0.009250,-0.017580,-61.1888,-38.9599,-58.1438,1.0,0.0,0.0,0.0
1,8.39,0,NaN,30.0,2.18837,8.56560,3.66179,2.39494,8.55081,3.64207,...,-0.006577,-0.004638,0.000368,-59.8479,-38.8919,-58.5253,1.0,0.0,0.0,0.0
2,8.40,0,NaN,30.0,2.37357,8.60107,3.54898,2.30514,8.53644,3.73280,...,0.003014,0.000148,0.022495,-60.7361,-39.4138,-58.3999,1.0,0.0,0.0,0.0
3,8.41,0,NaN,30.0,2.07473,8.52853,3.66021,2.33528,8.53622,3.73277,...,0.003175,-0.020301,0.011275,-60.4091,-38.7635,-58.3956,1.0,0.0,0.0,0.0
4,8.42,0,NaN,30.0,2.22936,8.83122,3.70000,2.23055,8.59741,3.76295,...,0.012698,-0.014303,-0.002823,-61.5199,-39.3879,-58.2694,1.0,0.0,0.0,0.0


Activity IDs – 1 lying – 2 sitting – 3 standing – 10 walking very slow – 11 normal walking – 12 Nordic walking – 13 running – 14 ascending stairs – 15 descending stairs – 16 cycling – 20 ironing – 21 vacuum cleaning – 22 rope jumping – 23 playing soccer – 0 other (transient activities)

In [27]:
ACTIVITIES_MAP_PAMAP = {
    0: 'no_activity',
    1: 'lying',
    2: 'sitting',
    3: 'standing',
    10: 'slow_walking',
    11: 'normal_walking',
    12: 'nordic_walking',
    13: 'running',
    14: 'ascending_stairs',
    15: 'descending_stairs',
    16: 'cycling',
    20: 'ironing',
    21: 'vaccuum_cleaning',
    22: 'rope_jumping',
    23: 'playing_soccer'
}

In [28]:
# lying	0.136615
# sitting	0.130736
# standing	0.136703
# walking	0.176625
# cycling	0.118540
# vaccuum_cleaning	0.125208
# ironing 
included_activities = [1, 2, 3, 11, 16, 21, 20]
exclude_activities = [n for n in ACTIVITIES_MAP_PAMAP if n not in included_activities]

In [29]:
def map_class_pamap(datasets_filled, exclude_activities):
    ysetall = [set(np.array(data.activityID)) - set(exclude_activities)
               for data in datasets_filled]
    class_ids = list(set.union(*[set(y) for y in ysetall]))
    class_labels = [ACTIVITIES_MAP_PAMAP[i] for i in class_ids]
    nr_classes = len(class_ids)
    mapclasses = {class_ids[i]: i for i in range(len(class_ids))}
    return class_labels, nr_classes, mapclasses

In [30]:
def preprocess(targetdir, outdatapath, columns_to_use, exclude_activities, fold,
               val_test_size=None):
    """ Function to preprocess the PAMAP2 data after it is fetched
    Parameters
    ----------
    targetdir : str
        subdirectory of directory_to_extract_to, targetdir
        is defined by function fetch_data
    outdatapath : str
        a subdirectory of directory_to_extract_to, outdatapath
        is the direcotry where the Numpy output will be stored.
    columns_to_use : list
        list of column names to use
    exclude_activities : list or tuple
        activities to exclude from the
    fold : boolean
        Whether to store each fold seperately ('False' creates
        Train, Test and Validation sets)
    Returns
    -------
    None
    """
    datadir = os.path.join(targetdir, 'PAMAP_Dataset', 'All')
    filenames = os.listdir(datadir)
    filenames.sort()
    print('Start pre-processing all ' + str(len(filenames)) + ' files...')
    # load the files and put them in a list of pandas dataframes:
    datasets = [pd.read_csv(os.path.join(datadir, fn), header=None, sep=' ')
                for fn in filenames]
    header = get_header_pamap()
    for i in range(0, len(datasets)):
        datasets[i].columns = header
    # Interpolate dataset to get same sample rate between channels
    datasets_filled = [d.interpolate() for d in datasets]
    # Create mapping for class labels
    class_labels, nr_classes, mapclasses = map_class_pamap(datasets_filled, exclude_activities)
    # Save class labels
    with open(os.path.join(outdatapath, 'labels.json'), 'w') as fp:
        json.dump(class_labels, fp)
    # Create input (x) and output (y) sets
    xall = [np.array(data[columns_to_use]) for data in datasets_filled]
    yall = [np.array(data.activityID) for data in datasets_filled]
    xylists = [tutorial_pamap2.split_activities(y, x, exclude_activities) for x, y in zip(xall, yall)]
    Xlists, ylists = zip(*xylists)
    ybinarylists = [tutorial_pamap2.transform_y(y, mapclasses, nr_classes) for y in ylists]
    frame_length = int(5.12 * 100)
    step = 1 * 100
    if not fold:
        if val_test_size is None:
            # Split in train, test and val
            x_vallist, y_vallist = tutorial_pamap2.split_data(Xlists, ybinarylists, indices=6)
            test_range = slice(7, len(datasets_filled))
            x_testlist, y_testlist = tutorial_pamap2.split_data(Xlists, ybinarylists, test_range)
            x_trainlist, y_trainlist = tutorial_pamap2.split_data(Xlists, ybinarylists,
                                                  indices=slice(0, 6))
            # Take sliding-window frames, target is label of last time step,
            # and store as numpy file
            x_train, y_train = tutorial_pamap2.sliding_window(frame_length, step, x_trainlist,
                                              y_trainlist)
            x_val, y_val = tutorial_pamap2.sliding_window(frame_length, step, x_vallist,
                                              y_vallist)
            x_test, y_test = tutorial_pamap2.sliding_window(frame_length, step, x_testlist,
                                              y_testlist)

        else:
            val_size, test_size = val_test_size
            X_list, y_list = tutorial_pamap2.split_data(Xlists, ybinarylists,
                                        slice(0, len(datasets_filled)))
            X, y = tutorial_pamap2.sliding_window(frame_length, step, X_list,
                                  y_list)
            x_train, y_train, x_val, y_val, x_test, y_test = tutorial_pamap2.split_data_random(X, y, val_size, test_size)


        tutorial_pamap2.numpify_and_store(x_train, y_train, X_name='X_train', y_name='y_train',
                            outdatapath=outdatapath, shuffle=True)
        tutorial_pamap2.numpify_and_store(x_val, y_val, X_name='X_val', y_name='y_val',
                            outdatapath=outdatapath, shuffle=False)
        tutorial_pamap2.numpify_and_store(x_test, y_test, X_name='X_test', y_name='y_test',
                            outdatapath=outdatapath, shuffle=False)
    else :
        for i in range(len(Xlists)):
            X_i, y_i = tutorial_pamap2.split_data(Xlists, ybinarylists, i)
            X, y = tutorial_pamap2.sliding_window(frame_length, step, X_i,
                                              y_i)
            tutorial_pamap2.numpify_and_store(X, y, X_name='X_'+str(i), y_name='y_'+str(i),
                            outdatapath=outdatapath, shuffle=True)


    print('Processed data succesfully stored in ' + outdatapath)
    return None

In [31]:
columns_to_use = ['hand_acc_x', 'hand_acc_y', 'hand_acc_z',
                 'ankle_acc_x', 'ankle_acc_y', 'ankle_acc_z',
                 'chest_acc_x', 'chest_acc_y', 'chest_acc_z']

In [32]:
preprocess(targetdir, outputdir, 
           columns_to_use, exclude_activities=exclude_activities, fold=False, val_test_size=(3000, 3000))

Start pre-processing all 15 files...
Stored /media/sf_VBox_Shared/timeseries/PAMAP_Dataset/cleaned_7act/X_train y_train
Stored /media/sf_VBox_Shared/timeseries/PAMAP_Dataset/cleaned_7act/X_val y_val
Stored /media/sf_VBox_Shared/timeseries/PAMAP_Dataset/cleaned_7act/X_test y_test
Processed data succesfully stored in /media/sf_VBox_Shared/timeseries/PAMAP_Dataset/cleaned_7act


In [33]:
x_train, y_train_binary, x_val, y_val_binary, \
           x_test, y_test_binary, labels = tutorial_pamap2.load_data(outputdir)

In [34]:
x_train.shape, y_train_binary.shape

((4400, 512, 9), (4400, 7))

In [35]:
x_test.shape

(3000, 512, 9)